<a href="https://colab.research.google.com/github/Droushb/NULP_NLP/blob/main/LPNLP_08_Text_generation_(2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Генерація тексту

У цій роботі ми використаємо мовну модель для генерації тексту.

У класичної мовної моделі є два взаємопов'язані визначення:

1. Оцінити ймовірність вхідного тексту.
2. Видати ймовірнісний розподіл наступного слова для даного префіксу.

Для генерації тексту нам ідеально підходить друге визначення.

## Початок роботи

Будь ласка, заповніть поля `EMAIL`, `NAME` та `GROUP` нижче:

In [40]:
!pip install --quiet --ignore-installed http://nlp.band/static/pypy/lpnlp-2023.10.2-py3-none-any.whl

In [42]:
################################################################################
# FILL-IN:
#-----------------------------------------------------------------------
EMAIL = "bohdan.drushkevych.kn.2021@lpnu.ua"  # заповніть вашим значенням
################################################################################

import lpnlp

lab = lpnlp.start(email=EMAIL, lab="text_generation")

Удачі!


### Завантаження моделі

Тренування мовної моделі з нуля займає багато часу: від кількох годин або днів для маленьких та середніх моделей й аж до кількох місяців чи навіть років для великих. Звичайно, за рахунок розпаралелювання тренування на багатьох GPU, реальний час рідко буває більше місяця-двох.

Для цієї роботи (як і в реальному житті), ми візьмемо претреновану модель. Хтось натренував її за нас. В данному випадку, візьмемо GPT-2 від компанії OpenAI.

Ця модель, як і безліч інших, зберігається на [HuggingFace Models](https://huggingface.co/models). Завантажити та працювати з нею зручно через бібліотеку [HuggingFace Transformers](https://github.com/huggingface/transformers/)

In [ ]:
!pip install transformers

In [ ]:
import transformers
import torch

In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained("gpt2")

## Токенізація

Кожна модель має словник токенів, з яким вона тренувалася, та правила токенізації (розбиття тексту на токени). Обов'язково слід використовувати той самий словник та метод.

Бібліотека transformers вміє робити це для кожної підтримуваної моделі.

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("gpt2")

Перевіримо токенізатор:

In [ ]:
input_text = "Hello, LP NLP!"
token_ids = tokenizer.encode(input_text)
token_ids

In [ ]:
tokenizer.convert_ids_to_tokens(token_ids)

Рідкісні слова будуть розбиті на підслова:

In [ ]:
tokenizer.convert_ids_to_tokens(tokenizer.encode("My name is Oleksiy Syvokon"))

## Перевірка моделі -- один крок ітерації

Ми генеруватимемо текст в циклі токен за токеном, зліва направо. Але для початку розберімо, як виглядає один крок такого циклу.


На кожному кроці на вхід моделі подаємо ті токени, які вже згенеровано.

На першому кроці у нас ще нічого не згенеровано. Тому починаємо зі токену `<BOS>` ("begin of sentence"). В різних моделях він виглядає по-різному. Подивимося, що в GPT-2:


In [ ]:
tokenizer.bos_token

In [ ]:
tokenizer.bos_token_id

In [ ]:
# Формуємо вхідний батч. Майже завжди для більшої ефективності
# нейронні мережі очікують на вхід кілька незалежних речень
# (або зображень у випадку з комп'ютерним зором)
#
# В нашій роботі ми завжди працюємо лише з одним реченням,
# тож розмір батча дорівнює одинці. Але все одно маємо
# оформити вхід як матрицю:
input_ = torch.LongTensor([[tokenizer.bos_token_id]])
input_

In [ ]:
# Нарешті робимо крок генерації
output = model(input_, return_dict=True)

# На виході модель повертає вектор з logits -- ненормалізованими
# ймовірностями кожного токена в словнику
output.logits.shape

Чому маємо саме таку розмірність?

* Перша одиниця -- розмір батча, тобто кількість вхідних речень.
* Друга одиниця -- це довжина вхідної послідовності. На першому кроці ми подали лише один токен (bos_token)
* 50257 -- це розмір словника

In [ ]:
tokenizer.vocab_size

Поки що модель повернула не ймовірності, а просто якісь числа. Їм треба нормалізувати функцією softmax:

In [ ]:
probs = torch.softmax(output.logits, dim=-1)

In [ ]:
# Розмірність має збігатися з розміром словника
probs.shape

In [ ]:
# Сума ймовірностей має дорівнювати 1.0
probs.sum()

Ймовірності можуть ставати дуже маленькими та причиняти проблеми у зв'язку з обмеженою точністю float чисел. Тому прийнято працювати з логарифмами ймовірностей.

In [ ]:
log_probs = torch.log_softmax(output.logits, dim=-1)

# Щоб перейти до звичайних ймовірностей, маємо зробити експоненціювання
log_probs.exp().sum()

Кожному слову в словнику відповідає своя ймовірність бути побаченим
після заданого префікса. Префіксом у нас поки що був лише одни `bos_token`.


In [ ]:
# Яка ймовірність, що речення почнеться зі слів "I", "red", "Why"?
for word in ("I", "red", "Why"):
    index = tokenizer.convert_tokens_to_ids(word)
    prob = log_probs[0, 0, index].exp()
    print(f"P({word}) = {prob}")

Тепер ми маємо ймовірностний розподіл по словнику. Можемо обрати слово, яке вважатимемо згенерованим. Тут можливі кілька стратегій, які ми розглянемо в наступних розділах.

## Greedy decoding

Найпростіший (але й не дуже цікавий) спосіб -- це завжди обирати токен з найбільшою ймовірністю:

In [ ]:
next_token_id = log_probs.argmax()
next_token_id

In [ ]:
probs[-1, -1, 198]

In [ ]:
tokenizer.convert_ids_to_tokens([next_token_id])  # Наш перший згенерований токен

Зберемо код докупи та додамо цикл. В циклі ми продовжуватимемо генерувати текст токен за токеном, поки не настане одна з двох умов:
1. Модель видала спецальний токен `eos_token` (end of sentence)
2. Довжина згенерованого тексту перевищила певний поріг `max_len`

У хорошої моделі в більшості випадків має спрацьовувати перша умова зупинки. Проте іноді модель може впасти в безкінчений цикл. Щоб цьому запобігти, маємо другу умову.

In [ ]:
@torch.no_grad()
def greedy_decode(model, tokenizer, max_len=50):
    start_index = tokenizer.bos_token_id
    result = [start_index]

    while len(result) < max_len:

        # Передбачення ймовірностней наступного токена
        input_ = torch.LongTensor([result])
        output = model(input_)
        probs = torch.log_softmax(output.logits[0, -1], dim=-1)

        # Обираємо токен, що має найбільшу ймовірність
        token_index = probs.argmax()

        # Зупиняємося, якщо досягли кінця
        if token_index == tokenizer.eos_token_id:
            break

        # Додаємо обраний токен в згенерований текст
        result.append(token_index.item())

    return result


generated_token_ids = greedy_decode(model, tokenizer)

Маємо список згенерованих індексів токенів, який починається ось так:

In [ ]:
generated_token_ids[:10]

Перетворимо їх в текст:

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(generated_token_ids)
tokens[:10]

In [ ]:
generated_text = "".join(tokens)
generated_text

In [ ]:
lab.checkpoint("greedy decode", generated_text)

### Примітка: Byte-pair encoding (BPE)

Наша модель використовує subword токенізацію, а саме byte-pair encoding (BPE). В сучасному NLP це найрозповсюдженіший спосіб токенізації. Детально можете подивитися в [цьому відео](https://www.youtube.com/watch?v=tOMjTCO0htA).

Для наших цілей зараз важливо, що BPE заміняє пробіли на спеціальні Unicode-символи "Ġ". Серед інших моделей широко поширений варіант "▁" (зверніть увагу, це не звичайний символ підкреслення "_"). Щоб отримати чистий текст, треба виконати наступну заміну:

In [ ]:
def bpe_decode(s):
    result = s.replace("Ġ", " ")
    result = result.replace("Ċ", "\n")
    return result

bpe_decode(generated_text)

Але краще довіритися токенізатору й зробити цю роботу за нас:

In [ ]:
tokenizer.convert_tokens_to_string(tokens)

## Generic decoding function

Обирати слово з найбільшою ймовірністю -- не найкращий варіант для генерації тексту хоча б тому, що він завжди детерміновано призводить до однієї послідовності. Нижче ми подивимося на цікавіші альтернативи.

Цикл генерації залишиться той самий, що і в `greedy_decode()`. Відрізнятися буде лише один рядок -- той, в якому ми приймали рішення, яке слово обрати. Для зручності, винесемо цей рядок в окрему функцію. Ця функція прийматиме на вхід ймовірностний розподіл по словнику і повертає обраний токен.

In [ ]:
def greedy_choice(probs):
    return probs.argmax()

Функцію генерації також трохи переробимо.

По-перше, додамо параметр `sample_fn` -- це має бути функція, яка обирає слово з ймовірностного розподілу, наприклад, `greedy_choice`.

По-друге, для зручності виконуватимемо BPE декодинг у середині функції генерації.

In [ ]:
@torch.no_grad()
def generate(model, tokenizer, sample_fn, max_len=50, bpe_decode=True):
    start_index = tokenizer.bos_token_id
    result = [start_index]

    while len(result) < max_len:

        # Передбачення ймовірностней наступного токена
        input_ = torch.LongTensor([result])
        output = model(input_)
        log_probs = torch.log_softmax(output.logits[0, -1], dim=-1)

        # Обираємо токен, що має найбільшу ймовірність
        token_index = sample_fn(log_probs.exp())     # <---------------- цей рядок змінено

        # Зупиняємося, якщо досягнули кінця
        if token_index == tokenizer.eos_token_id:
            break

        # Додаємо обраний токен в згенерований текст
        result.append(token_index.item())

    if bpe_decode:
        tokens = tokenizer.convert_ids_to_tokens(result)
        result = tokenizer.convert_tokens_to_string(tokens)

    return result


generate(model, tokenizer, greedy_choice)

## Simple sampling

Перший альтернатива -- це sampling. Тепер ми обираємо наступний токен випадково, але пропорційно до ймовірностей.

In [ ]:
def simple_sample(probs):
    return torch.multinomial(probs, num_samples=1)[0]

# Згенеруємо 5 речень
for i in range(1, 6):
    result = generate(model, tokenizer, simple_sample)
    print(f"#{i}: {result}")
    print()

## Sampling with temperature

Ми також можемо впливати на генерацію параметром температури softmax.

Більші значення температури призводять до того, що різниця між ймовірностями токенів зменшується, тобто розподіл стає більш рівномірним. На практиці це означає, що менш ймовірні варіанти обиратимуться частіше і згенерований текст може бути цікавішим. Однак якщо продовжувати піднімати температуру, то текст спочатку втратить зв'язність, далі почнуть розпадатися слова та граматичність.

Менші значення температури змінюють розподіл таким чином, що основна ймовірніста маса припадає на невелику кількість топових токенів. При температурі 0 вся ймовірність дістанеться одному токену й семплінг перетвориться на greedy decoding.

Згенеруємо тексти з різною температурою:

In [45]:
for temperature in (0.1, 0.3, 0.5, 0.8, 1.0, 1.25, 1.5, 2.0, 3.0, 5.0):

    def sample_with_temp(probs):
        updated_probs = probs.log().div(temperature).exp()
        return simple_sample(updated_probs)

    print(f"Sampling with temperature={temperature}")
    result = generate(model, tokenizer, sample_with_temp, max_len=15)
    print(result)
    print()

Sampling with temperature=0.1
<|endoftext|>
The U.S. Department of Justice has been investigating the use

Sampling with temperature=0.3
<|endoftext|>The world's largest data center is the world's largest data center,

Sampling with temperature=0.5
<|endoftext|>"I'm not sure what to say, but I'm just trying

Sampling with temperature=0.8
<|endoftext|>"It makes you sigh out loud: You're right. It makes

Sampling with temperature=1.0
<|endoftext|>Last Sunday, the vast majority of voters were not happy with the election

Sampling with temperature=1.25
<|endoftext|>'t V1 God lessons from Jake Liveseren at Cloud Imperium Games.

Sampling with temperature=1.5
<|endoftext|> 1925 notice showing street developements in Palit located off Hinds Rivers

Sampling with temperature=2.0
<|endoftext|>Wait Nothing Second Abs["Rockbanks Mysterious Sheriff Ron impeachment of dictator 95

Sampling with temperature=3.0
<|endoftext|>Asian greens recalling pins227 offer URIHyp literacy27oser Expressionaverage

In [43]:
# Яке значення `temperature` здається вам оптимальною?
lab.checkpoint("softmax temperature", 1.0)

Відповідь правильна ✅
Окей, добре


1.0

## Top-k sampling

In [46]:
def top_k_sampling(probs, k):
    topk = probs.topk(k)
    index = torch.multinomial(topk.values, num_samples=1)[0]
#     print(f"Top {k} words take {topk.values.sum():%} probability mass")
    return topk.indices[index]

In [47]:
k = 15
sample_fn = lambda probs: top_k_sampling(probs, k=k)
for i in range(1, 6):
    result = generate(model, tokenizer, sample_fn, max_len=20)
    print(f"#{i}: {result}")
    print()

#1: <|endoftext|>
The US has sent troops to support forces fighting Isis in the north-eastern town of

#2: <|endoftext|>This is a rush transcript. Copy may not be in its final form.

AMY GOODMAN

#3: <|endoftext|>A group of scientists has found evidence that a common, low level of stress may be linked to

#4: <|endoftext|>Bethany Moulins of the University of California at San Diego says that her husband's

#5: <|endoftext|>A new study by researchers from the U.S. Environmental Protection Agency (EPA) found that



## Nucleus (top-p) sampling

In [48]:
def nucleus_sampling(probs, max_p):
    sorted_probs = probs.sort(descending=True)
    cum_prob = 0.0
    sample_indices = []
    sample_probs = []
    for i in range(0, len(sorted_probs.values)):
        p = sorted_probs.values[i]
        cum_prob += p
        sample_probs.append(p)
        sample_indices.append(sorted_probs.indices[i])
        if cum_prob >= max_p:
            break

    index = torch.multinomial(torch.tensor(sample_probs), num_samples=1)
    return sample_indices[index]

In [49]:
for max_p in (0.0, 0.1, 0.3, 0.5, 0.6, 0.8, 1.0):
    sample_fn = lambda probs: nucleus_sampling(probs, max_p=max_p)
    result = generate(model, tokenizer, sample_fn, max_len=20)
    print(f"#{max_p}: {result}")
    print()

#0.0: <|endoftext|>
The first time I saw the new version of the game, I was so excited. I

#0.1: <|endoftext|>
The new report from the National Institute of Mental Health (NIMH) on mental health

#0.3: <|endoftext|>In the past, we've seen the rapid rise of these microservices. We've seen them

#0.5: <|endoftext|>8th August 2018

4th August 2018

6th August 2018

5

#0.6: <|endoftext|>In my last post, I talked about how if you go through the template with the STL,

#0.8: <|endoftext|>Note: This page is based on a project from the previous project.

Installation



#1.0: <|endoftext|>LAS VEGAS (CBS) – A 13-year-old Lansing girl left the



## Start from prompt

До цього моменту ми генерували текст з нуля. Однак значно кориснішим є задача генерації тексту від певного префікса або "підказки" -- в англійській мові це називається "prompt".

Prompt дозволить нам контролювати тематику згенерованого тексту або, як ми побачимо на іншій лекції, допоможе моделі виконувати різноманітні завдання.

In [50]:
@torch.no_grad()
def generate(model, tokenizer, sample_fn, prompt, max_len=50, bpe_decode=True):

    result = tokenizer.encode(prompt)     # <---- цей рядок додано

    while len(result) < max_len:

        # Передбачення ймовірностней наступного токена
        input_ = torch.LongTensor([result])
        output = model(input_)
        log_probs = torch.log_softmax(output.logits[0, -1], dim=-1)

        # Обираємо токен, що має найбільшу ймовірність
        token_index = sample_fn(log_probs.exp())

        # Зупиняємося, якщо досягнули кінця
        if token_index == tokenizer.eos_token_id:
            break

        # Додаємо обраний токен в згенерований текст
        result.append(token_index.item())

    if bpe_decode:
        tokens = tokenizer.convert_ids_to_tokens(result)
        result = tokenizer.convert_tokens_to_string(tokens)

    return result

In [51]:
k = 3
sample_fn = lambda probs: top_k_sampling(probs, k=k)
for i in range(1, 5):
    result = generate(model, tokenizer, sample_fn, prompt="One of my favorite things is")
    print(f"#{i}: {result}")
    print()


# Спробуйте змінити prompt на щось інше

#1: One of my favorite things is the way the game is presented. It's like a game that you can see and feel and it's very easy to play and very easy to play. It's very easy to play and very easy to play.


#2: One of my favorite things is the way the game is played. It's a very simple game. You can't really tell if you're going to win or lose, but you can see where you are. You can see what your opponent is doing

#3: One of my favorite things is the fact that I can go back and forth between them. I've been in the business for a while now, and my wife and I are very close. I've been in the business for about 10 years now.

#4: One of my favorite things is that it's so easy to get started with it. It's just so easy. It's like you're in a game of chess. It's like you're in an open world where there are lots of places to



# Zero-shot класифікація

Ми будемо досліджувати цю тему на наступній лекції. Але спробуємо застосувати мовну модель для задачі класифікації вже зараз. Хоча наша модель надто маленька й слабка для серйозного використання в zero-shot.


In [52]:
generate(model, tokenizer, greedy_choice, prompt="This film is awful! Actor play is terrible. The plot is dull. I would rate it as a").splitlines()[0]


'This film is awful! Actor play is terrible. The plot is dull. I would rate it as a B+.'

In [53]:
generate(model, tokenizer, greedy_choice, prompt="This film is amazing! The time flies by when you watch it. Definitely recommend! On the scale of 1 to 5, I would rate it as a").splitlines()[0]


'This film is amazing! The time flies by when you watch it. Definitely recommend! On the scale of 1 to 5, I would rate it as a 5.'

In [54]:
lab.answer("Готово!")

Відповідь правильна ✅
Ця робота не має однієї правильної відповіді. Вважаємо лабу пройденою :) Формочка: https://tally.so/r/mZ81ve


'Готово!'

# Real world

У цій роботі ми імплементували кілька методів декодінгу. Але, звичайно, все вже зроблено за нас.

Подивіться на параметри функції [generate()](https://huggingface.co/docs/transformers/v4.21.1/en/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate) з бібліотеки transformers. Багато з них мають виглядати знайомими.

Повний приклад використання:

In [55]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

prompt = "I hope that in that practical class on text generation you"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, do_sample=True, top_p=0.8, max_length=50)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["I hope that in that practical class on text generation you'll also be able to learn to draw with a pencil, and to make beautiful art on paper.\n\nMy name is Thomas and I am from California. I grew up in the Bay Area"]

# The End


In [56]:
# Сподіваюсь, ця лаба була не надто важкою :)
lab.answer("Готово!")

Відповідь правильна ✅
Ця робота не має однієї правильної відповіді. Вважаємо лабу пройденою :) Формочка: https://tally.so/r/mZ81ve


'Готово!'